<a href="https://colab.research.google.com/github/BirHic/Interpolation/blob/main/Internation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asena PLUG - Video İnerpolation.

In [ ]:
# Google Drive Bağla
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
################# Gerekli Yapılandırma ############################
 
#@markdown #Gerekli Yapılandırma
#@markdown Drive üzerinde "videos" klasöru açın. Yüklemek istediğiniz videoyu oraya yükleyin.
#@markdown ## Video İsmi
#@markdown Lütfen sadece `"kaplum"` kısmını silin ardından videonun ismini girin. Sadece `480p` ve `720p` videolar desteklenmektir.
INPUT_FILEPATH = "videos/kaplum.mp4" #@param{type:"string"}
 
#@markdown ## Sonuç Dosya İsmi
#@markdown Sadece `.mp4` ve `.gif` uzantıları geçerlidir. Lütfen sadece `"kaplum240fps"` kısmını değiştirin.
OUTPUT_FILE_PATH = "DAIN/kaplum240fps.mp4" #@param{type:"string"}
 
 
 
 
#@markdown ## Hedef FPS
#@markdown Videoyu ayarlamak istediğiniz FPS değerini girin. FPS değeri ne kadar yüksek olursa işlem o kadar uzun sürede tamamlanır.
TARGET_FPS =  240#@param{type:"number"}
 
#@markdown ###`↓BİLGİNİZ YOKSA DOKUNMAYIN↓`
 
 
#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/DAIN/input_frames' #@param{type:"string"}
 
#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' #@param{type:"string"}
 
#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = True #@param{type:"boolean"}
 
#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}
 
#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = False #@param{type:"boolean"}

# Modüller Yükleniyor. 
Bu işlem 10-15 dakika arası sürecektir.
Yükleme esnasında hiçbir paket bağlı olduğunu ağ üzerinden indirilmez.

In [ ]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install pytorch==1.1 cudatoolkit torchvision -c pytorch -y
!conda install ipykernel -y
 
!pip install scipy==1.1.0
!pip install imageio
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")
 
# Repo Yükleniyor
%cd /content
!git clone -b master --depth 1 https://github.com/baowenbo/DAIN /content/DAIN
%cd /content/DAIN
!git log -1
 
# Repo Oluşturuluyor
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")
 
# PyTorch Yükleniyor
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")
 
# Pre-trained Modeller Yükleniyor
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
print("Tamamlandı")

In [ ]:
# Videodaki FPS değeri ölçülüyor.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/
 
import os
filename = os.path.basename(INPUT_FILEPATH)
 
import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')
 
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Input file has {fps} fps")
 
if(fps/TARGET_FPS>0.5):
  print("LÜTFEN FPS DEĞERİNİ YÜKSELTİN. GİRİLEN FPS DEĞERİ YENİ KARELER YARATMAK İÇİN ÇOK DÜŞÜK.")

In [ ]:
# FFMPEG İLE YENİ KARELER ÜRETİLİYOR.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'
 
if (END_FRAME==-1):
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=gte(n\,{START_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'
else:
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=between(n\,{START_FRAME}\,{END_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'
 
from IPython.display import clear_output
clear_output()
 
png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
frame_count = int(png_generated_count_command_result.output.strip())
 
import shutil
if SEAMLESS:
  frame_count += 1
  first_frame = f"{FRAME_INPUT_DIR}/00001.png"
  new_last_frame = f"{FRAME_INPUT_DIR}/{frame_count.zfill(5)}.png"
  shutil.copyfile(first_frame, new_last_frame)
 
print(f"{frame_count} frame PNGs generated.")
 
#Checking if PNGs do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")
 
# Removing alpha if detected
if "a" in channels:
  print("Alpha channel detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [ ]:
# İnterpolate Ediliyor.
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'
print("Tamamlandı")

In [ ]:
# Video Oluşturuluyor.
import numpy as np
%cd {FRAME_OUTPUT_DIR}
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False#
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)
 
%cd /content/DAIN
print("Tamamlandı")

In [ ]:
# Video Kodlanıyor.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

if(AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*

# `ÖNEMLİ BİLGİ` 
### Sayfa yenilenirse yada bağlantı disconnect olursa işlemlere en baştan başlamanız gerekmektedir.

 ### Aşağıdaki seçenekler geliştiricileri içindir. İki seçenek de `çalışmamaktadır.` 
Geliştiriciler istediği gibi düzenleyebilir.

In [ ]:
# [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'
 
if (AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac